# Basic Aggregate Functions

## Percentage of Users Attended a Contest
#### Difficulty: $\star$
#### Hint: *use `select` inside aggregate function*
#### Problem:
Table: `Users`

| Column Name | Type    |
|-------------|---------|
| user_id     | int     |
| user_name   | varchar |

`user_id` is the primary key for this table.
Each row of this table contains the name and the id of a user.
 

Table: `Register`

| Column Name | Type    |
|-------------|---------|
| contest_id  | int     |
| user_id     | int     |

(`contest_id`, `user_id`) is the primary key for this table.
Each row of this table contains the id of a user and the contest they registered into.
 
Write an SQL query to find the percentage of the users registered in each contest rounded to **two decimals**. Return the result table ordered by **percentage in descending order**. In case of a tie, order it by **contest_id in ascending order**.

#### Solution
There is no need to join tables every time. In this case, we can just use `select` inside the aggregrat function, which will make it easier to find how many people in total. 

*Note: this problem can also be solved by left join, though the process is more complicated*

In [ ]:
Select contest_id, ROUND((COUNT(user_id) / (Select COUNT(user_id) from Users)) * 100, 2) AS percentage
From Register r
Group by contest_id
Order by percentage desc, contest_id

## Queries Quality and Percentage
#### Difficulty: $\star$
#### Hint: *conditional sum*
#### Solution
The only thing to pay attention to in this question is to count rows in a table that satisfy a certain situation. In this case, we can use a conditional count function:
```SQL
SUM(COUNT(CASE WHEN condition THEN 1 ELSE 0 END))
```
This means we only count a certain row when the conditional is met and continues otherwise. 

In [ ]:
Select 
    query_name, 
    ROUND((AVG(rating / position)), 2) AS quality,
    ROUND((SUM(CASE WHEN rating < 3 THEN 1 ELSE 0 END) * 100 / COUNT(rating)), 2) AS poor_query_percentage
From Queries 
Group by query_name

## Immediate Food Delivery II
#### Difficulty: $\star$
#### Hint: *subquery, `IN` operator*
#### Problem
Table: `Delivery`

| Column Name                 | Type    |
|-----------------------------|---------|
| delivery_id                 | int     |
| customer_id                 | int     |
| order_date                  | date    |
| customer_pref_delivery_date | date    |

`delivery_id` is the primary key of this table.
The table holds information about food delivery to customers that make orders at some date and specify a preferred delivery date (on the same order date or after it).
 

If the customer's preferred delivery date is the same as the order date, then the order is called **immediate**; otherwise, it is called **scheduled**.

The **first order** of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has **precisely one** first order.

Write an SQL query to find the percentage of immediate orders in the first orders of all customers, rounded to **2 decimal places**.

#### Solution
The key is to use a subquery to filter out all the first orders.
About the code:
1. The inner subquery retrieves the minimum order date for each customer by grouping the records in the `Delivery` table by `customer_id` and selecting the minimum order_date using the `min` function. 
2. The outer query then uses the result set from the inner subquery as a filter condition using the **IN** operator. It filters the records in the `Delivery` table based on matching `customer_id` and `order_date` with the minimum order date for each customer.

In [ ]:
SELECT
    ROUND(100*SUM(CASE WHEN order_date = customer_pref_delivery_date THEN 1 ELSE 0 END)/ COUNT(customer_id) ,2) AS immediate_percentage
FROM
    Delivery
WHERE
    (customer_id, order_date)
IN
(SELECT
    customer_id, min(order_date) as min_date
FROM
    Delivery
GROUP BY
    customer_id
);

## Game Play Analysis IV
#### Difficulty: $\star\star$
#### Hint: *Self joining subqueries*
#### Problem

Table: `Activity`

| Column Name  | Type    |
|--------------|---------|
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |

(`player_id`, `event_date`) is the primary key of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.
 

Write an SQL query to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for **at least two consecutive days** starting from **their first login date**, then divide that number by the total number of players.



In [ ]:
Select ROUND(COUNT(a2.player_id) / COUNT(a1.player_id), 2) AS fraction
From
  (Select player_id, MIN(event_date) AS first_login
  From Activity
  Group by player_id) a1
Left Join Activity a2
ON a1.player_id = a2.player_id and a1.first_login + 1 = a2.event_date